In [1]:
pwd

'/home/wbaik/Code/data_science/playground_pytorch/Notebooks'

In [2]:
cd ..

/home/wbaik/Code/data_science/playground_pytorch


In [3]:
import iex_utils

In [4]:
import pandas as pd
import numpy as np
from iex_utils import create_df_from_symbols

In [7]:
spx_table = pd.read_csv('./iexdata/10K_data.csv')
sectors_list = list(set(spx_table['Sector']))
by_sectors = spx_table.groupby('Sector')

In [8]:
sectors_list[0]

'Real Estate'

In [9]:
random_sector = by_sectors.get_group(sectors_list[0])['Symbol'][:30]
temp_df = create_df_from_symbols(*random_sector)

In [10]:
columns_to_drop = list(filter(lambda x: 'close' not in x, temp_df.columns))

In [11]:
temp_df.drop(columns_to_drop, axis=1, inplace=True)

In [12]:
from compute_ratios import heatmap
from collections import defaultdict
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [97]:
def some_strategy(prices, lb, hold):
    
    def get_portfolio_logic(price, lookback, lag):
        momentum = price.shift(lag).pct_change(lookback)
        ranks = momentum.rank(axis=1, ascending=False)
        best_ten = lambda x: list(map(lambda y: y 
                                      if y > price.shape[1] - 10 
                                      else 0, x))
        ranks = ranks.apply(best_ten, axis=1)
        return ranks
    
    compound = lambda x: (1 + x).prod() - 1
    daily_sr = lambda x: x.mean() / x.std()
    
    # Compute portfolio weights
    freq = '%dB' % hold
    port = get_portfolio_logic(prices, lb, lag=0)
    # Normalize
    port = port.apply(lambda x: x/x.sum(), axis=1)
    
    daily_rets = prices.pct_change()

    # `resample` merely loops over for each freq
    # Takes the first value of the resampled
    port = port.shift(1).resample(freq).first() # shift 1 for trading at the close
    returns = daily_rets.resample(freq).apply(compound)
    port_rets = (port * returns).sum(axis=1)
    port_rets.name = 'lookback:{}, hold:{}'.format(str(lb), str(hold))

    return daily_sr(port_rets) * np.sqrt(252 / hold), port_rets

In [98]:
from functools import reduce

In [103]:
lookbacks = range(50, 100, 10)
holdings = range(50, 100, 10)
dd = defaultdict(dict) # dictionary of dictionary

ret_d = []
for lb in lookbacks:
    for hold in holdings:
        dd[lb][hold], temp = some_strategy(temp_df, lb, hold)
        ret_d.append(temp)
    
ret_d = reduce(lambda x, y: pd.concat([x, y], axis=1), ret_d)


ddf = pd.DataFrame(dd)
ddf.index.name = 'Holding Period'
ddf.columns.name = 'Lookback Period'

In [109]:
test = ret_d.fillna(0)

In [110]:
test.apply(lambda x: x.cumsum())

,"lookback:50, hold:50","lookback:50, hold:60","lookback:50, hold:70","lookback:50, hold:80","lookback:50, hold:90","lookback:60, hold:50","lookback:60, hold:60","lookback:60, hold:70","lookback:60, hold:80","lookback:60, hold:90",...,"lookback:80, hold:50","lookback:80, hold:60","lookback:80, hold:70","lookback:80, hold:80","lookback:80, hold:90","lookback:90, hold:50","lookback:90, hold:60","lookback:90, hold:70","lookback:90, hold:80","lookback:90, hold:90"
date,,,,,,,,,,,,,,,,,,,,,
2013-05-06,0.000000,-0.125719,-0.151272,-0.189432,-0.204423,0.000000,0.000000,-0.165935,-0.204323,-0.215678,...,0.000000,0.000000,0.000000,0.000000,-0.219886,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-15,-0.055832,-0.125719,-0.151272,-0.189432,-0.204423,-0.071823,0.000000,-0.165935,-0.204323,-0.215678,...,-0.074235,0.000000,0.000000,0.000000,-0.219886,-0.074457,0.000000,0.000000,0.000000,0.000000
2013-07-29,-0.055832,-0.153760,-0.151272,-0.189432,-0.204423,-0.071823,-0.030687,-0.165935,-0.204323,-0.215678,...,-0.074235,-0.046523,0.000000,0.000000,-0.219886,-0.074457,-0.045959,0.000000,0.000000,0.000000
2013-08-12,-0.055832,-0.153760,-0.163410,-0.189432,-0.204423,-0.071823,-0.030687,-0.159244,-0.204323,-0.215678,...,-0.074235,-0.046523,-0.033156,0.000000,-0.219886,-0.074457,-0.045959,-0.033568,0.000000,0.000000
2013-08-26,-0.055832,-0.153760,-0.163410,-0.211983,-0.204423,-0.071823,-0.030687,-0.159244,-0.200360,-0.215678,...,-0.074235,-0.046523,-0.033156,-0.018740,-0.219886,-0.074457,-0.045959,-0.033568,-0.020016,0.000000
2013-09-09,-0.055832,-0.153760,-0.163410,-0.211983,-0.179449,-0.071823,-0.030687,-0.159244,-0.200360,-0.185728,...,-0.074235,-0.046523,-0.033156,-0.018740,-0.189203,-0.074457,-0.045959,-0.033568,-0.020016,0.035298
2013-09-23,-0.105392,-0.153760,-0.163410,-0.211983,-0.179449,-0.131604,-0.030687,-0.159244,-0.200360,-0.185728,...,-0.117500,-0.046523,-0.033156,-0.018740,-0.189203,-0.121588,-0.045959,-0.033568,-0.020016,0.035298
2013-10-21,-0.105392,-0.150321,-0.163410,-0.211983,-0.179449,-0.131604,-0.029626,-0.159244,-0.200360,-0.185728,...,-0.117500,-0.075343,-0.033156,-0.018740,-0.189203,-0.121588,-0.048896,-0.033568,-0.020016,0.035298
2013-11-18,-0.105392,-0.150321,-0.064084,-0.211983,-0.179449,-0.131604,-0.029626,-0.075083,-0.200360,-0.185728,...,-0.117500,-0.075343,0.048703,-0.018740,-0.189203,-0.121588,-0.048896,0.053816,-0.020016,0.035298


In [49]:
import os
return_df = os.listdir('./iexdata/')

In [51]:
return_df = list(filter(lambda x: 'return_df' in x, return_df))

In [52]:
return_df

['Health Care_top_10_long_only_return_df',
 'Real Estate_top_10_long_only_return_df',
 'Consumer Discretionary_top_10_long_only_return_df',
 'Telecommunication Services_top_10_long_only_return_df',
 'Industrials_top_10_long_only_return_df',
 'Utilities_top_10_long_only_return_df',
 'Energy_top_10_long_only_return_df',
 'Information Technology_top_10_long_only_return_df',
 'Materials_top_10_long_only_return_df',
 'Financials_top_10_long_only_return_df',
 'Consumer Staples_top_10_long_only_return_df']

In [68]:
from itertools import islice

def generate_best_returned_df(given_sector, num_best):
    sector = pd.read_csv('./iexdata/{}'.format(given_sector))
    sector.set_index('date', inplace=True, drop=True)
    
    largest_return_indecies = reversed(sector.iloc[-1,:].values.argsort())
    best_returns = list(islice(largest_return_indecies, num_best))
    
    return sector.take(best_returns, axis=1)

In [69]:
generated_dfs = map(lambda x: generate_best_returned_df(x, 8), return_df)

In [70]:
for df, sector in zip(generated_dfs, return_df):
    sector_part = sector.split('_')[0]
    df.to_csv('./iexdata/{}_best_returns'.format(sector_part))

In [74]:
return_files = list(filter(lambda x: 'best_returns' in x, os.listdir('./iexdata/')))

In [75]:
temp = pd.read_csv('./iexdata/{}'.format(return_files[0]))

In [76]:
temp

,date,"lookback:100, hold:100","lookback:110, hold:100","lookback:60, hold:60","lookback:110, hold:90","lookback:100, hold:90","lookback:110, hold:80","lookback:30, hold:60","lookback:40, hold:60"
0,2013-05-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.019466,-0.021795
1,2013-06-17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.019466,-0.021795
2,2013-07-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.019466,-0.021795
3,2013-07-15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.019466,-0.021795
4,2013-07-29,0.000000,0.000000,0.099147,0.000000,0.000000,0.000000,0.066976,0.049302
5,2013-08-12,0.000000,0.000000,0.099147,0.000000,0.000000,0.000000,0.066976,0.049302
6,2013-08-26,0.000000,0.000000,0.099147,0.000000,0.000000,0.101301,0.066976,0.049302
7,2013-09-09,0.000000,0.000000,0.099147,0.191178,0.192034,0.101301,0.066976,0.049302
8,2013-09-23,0.131540,0.130489,0.099147,0.191178,0.192034,0.101301,0.066976,0.049302
9,2013-10-07,0.131540,0.130489,0.099147,0.191178,0.192034,0.101301,0.066976,0.049302
